In [1]:
#Question2
!pip install datasets
from datasets import load_dataset

import pandas as pd
import re
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, pipeline

# Load CSVs
khalid_df = pd.read_csv('/content/Khalid.csv')
gaga_df = pd.read_csv('/content/LadyGaga.csv')

# Combine them
lyrics_df = pd.concat([khalid_df, gaga_df])

# Clean the lyrics
def clean_lyrics(lyric):
    if pd.isna(lyric):
        return ""
    lyric = str(lyric)
    lyric = re.sub(r'^#+', '', lyric)  # remove leading hashes
    lyric = lyric.encode('utf-8').decode('utf-8', 'ignore')  # remove weird chars
    lyric = re.sub(r'[\u2018\u2019\u201c\u201d]+', "'", lyric)  # smart quotes to '
    lyric = re.sub(r'[^\x00-\x7F]+', '', lyric)  # remove non-ascii (optional)
    return lyric.strip()

# Apply cleaning and extract as list
lyrics_texts = lyrics_df['Lyric'].dropna().apply(clean_lyrics).tolist()

# Save cleaned lyrics to a text file
with open("lyrics_dataset.txt", "w", encoding="utf-8") as f:
    for lyric in lyrics_texts:
        f.write(lyric + "\n\n")

# Load dataset from text
dataset = load_dataset("text", data_files={"train": "lyrics_dataset.txt"})

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Tokenize data
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define training args
training_args = TrainingArguments(
    output_dir="./gpt2-lyrics",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=100,
    save_steps=500,
    save_total_limit=1,
    prediction_loss_only=True,
    report_to="none",  # 🚫 Disable W&B
    fp16=False
)


# Setup Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
)

# Train the model
# The labels are the same as the input_ids in language modeling
tokenized_dataset = tokenized_dataset.map(lambda examples: {'labels': examples['input_ids']}, batched=True)
trainer.train_dataset = tokenized_dataset["train"]
trainer.train()

# Generate text from trained model
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
print(generator("I remember those nights when", max_length=100, num_return_sequences=1)[0]["generated_text"])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Map:   0%|          | 0/918 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.037900
200,0.900600
300,0.814400
400,0.761700
500,0.788100
600,0.752900
700,0.676300
800,0.689400
900,0.761700
1000,0.639500


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


I remember those nights when he cried me just you and me would dance and when we kissed our lips underneath the rainbow light 'cause we kissed and we mumbled we sang in all the way oh we sang and he was our boyfriend ever since he told us that he could love 'em like a man and i still love them like a man   if you love them they'll never know the pain of you no matter how small or what they'll do 'cause they know they know you know you
